In [1]:
with open("./data/example10.txt", "r") as f:
    example = f.read()

In [2]:
with open('./data/day10.txt', 'r') as f:
    data = f.read()

In [3]:
example

'[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}\n[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}\n[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}'

In [28]:
import re

def parse_input(input_str):
    lights = []
    switches = []
    randoms = []
    for line in input_str.splitlines():
        sections = line.split(' ')
        light = list(sections[0].strip('[').strip(']'))
        switch = []
        for section in sections[1:-1]:
            numbers = re.findall(r"\(([\d,]+)\)", section)
            if numbers:
                num_list = tuple(int(n) for n in numbers[0].split(","))
            switch.append(num_list)
        random = sections[-1]
        lights.append(light)
        switches.append(switch)
        randoms.append(random)
    return lights, switches, randoms

In [29]:
lights, switches, randoms = parse_input(example)

In [ ]:
def min_steps(current_lights, switches, path: list):
    for switch in switches:
        cur_path = path.copy()
        cur_path.append(switch)
        new_lights = current_lights.copy()
        for light in switch:
            if new_lights[light] == ".":
                new_lights[light] = "#"
            else:
                new_lights[light] = "."

In [31]:
for i in min_steps(lights[0], switches[0], []):
    print(i)

(['.', '#', '#', '#'], [(3,)])
(['.', '.', '#', '#'], [(1, 3)])
(['.', '#', '.', '.'], [(2,)])
(['.', '#', '.', '#'], [(2, 3)])
(['#', '#', '.', '.'], [(0, 2)])
(['#', '.', '#', '.'], [(0, 1)])


In [30]:
lights

[['.', '#', '#', '.'],
 ['.', '.', '.', '#', '.'],
 ['.', '#', '#', '#', '.', '#']]